In [ ]:
# Django notebook kernel must be running
# python manage.py shell_plus --notebook

In [ ]:
import django_initialiser
import time
import numpy as np

In [ ]:
from api.models import Song
from api.melody_utils.generator import generate_all_piano_midi_de

In [ ]:
# generate_all_piano_midi_de("../midi/piano_midi_de")

In [ ]:
bach = Song.objects.filter(
    author__contains='bach'
)
elise = Song.objects.filter(
    title__contains='elise'
)
# print(bach[0].note_sequence)
print(list(map(int, elise[0].note_sequence.split(','))))

In [ ]:
# Melodies expected as lists
def longest_substring(song, query):
    # find the length of the strings
    m = len(song)
    n = len(query)

    # declaring the array for storing the dp values
    a = np.zeros(shape=[m + 1, n + 1], dtype='int')

    longest = 0

    # LCS DP
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                a[i][j] = 0

            elif song[i - 1] == query[j - 1]:
                a[i][j] = a[i - 1][j - 1] + 1
                longest = max(longest, a[i][j])

            else:
                a[i][j] = 0

    return longest

In [ ]:
def lookup(query):
    t = time.process_time()
    # Retrieve all songs from db
    songs = Song.objects.all()

    results = []

    for song in songs:
        song_notes = list(map(int, song.note_sequence.split(',')))
        res = longest_substring(song_notes, query)
        results.append((res, song))

    # Sort in descending order by the longest subsequence length
    results.sort(key=lambda tup: tup[0], reverse=True)
    t = time.process_time() - t
    print(f'Done in {t} seconds')
    return results

In [ ]:
input_search = [76, 75, 76, 75, 76, 71, 74, 72, 69, 57]
elise_notes = list(map(int, elise[0].note_sequence.split(',')))
print(longest_substring(elise_notes, input_search))

In [ ]:
res = lookup(input_search)
for i in range(0, 3):
    print(res[i][0], res[i][1].title)